In [58]:
#Import Library
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import time
import pandas as pd
import seaborn as sb


In [32]:
property_data = pd.read_csv("NY property data.csv")

property_data

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070989,1070990,5080500083,5,8050,83,NaN,"TOBIN, GALE",A1,1,60,...,30,26,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR
1070990,1070991,5080500086,5,8050,86,NaN,SHERRI MILINAZZO,A1,1,62,...,32,26,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR
1070991,1070992,5080500089,5,8050,89,NaN,JOHN GERVASI,A1,1,53,...,24,32,NaN,NaN,NaN,NaN,1017.0,FINAL,2010/11,AC-TR
1070992,1070993,5080500092,5,8050,92,NaN,RITA M MOOG,A1,1,52,...,32,21,NaN,NaN,NaN,NaN,NaN,FINAL,2010/11,AC-TR


In [33]:
# Filling in missing ZIP

missing_zips = np.where(pd.isnull(property_data["ZIP"]))[0]
len(missing_zips)

29890

In [34]:
for i in range(len(missing_zips)):
    if(property_data.loc[missing_zips[i]+1,"ZIP"] == property_data.loc[missing_zips[i] - 1, "ZIP"]):
        property_data.loc[missing_zips[i], "ZIP"] = property_data.loc[missing_zips[i] - 1, "ZIP"]

In [35]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

16437

In [36]:
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i], "ZIP"] = property_data.loc[missing_zips[i] - 1, "ZIP"]

In [37]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

In [38]:
#filling in missing stories
temp = property_data[property_data["STORIES"].isnull()]
len(temp)

56264

In [40]:
temp["TAXCLASS"].value_counts()

1B    24736
4     22354
3      4635
2      3435
1       897
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [41]:
mean_stories = property_data.groupby("TAXCLASS")["STORIES"].mean()
print(mean_stories)

TAXCLASS
1      2.111641
1A     1.656837
1B     4.000000
1C     3.052748
1D     1.068966
2     16.095110
2A     2.844574
2B     4.004494
2C     4.745097
3      1.333333
4      5.446968
Name: STORIES, dtype: float64


In [42]:
len(property_data[property_data["STORIES"] == 0])

0

In [44]:
property_data["STORIES"] = property_data["STORIES"].fillna(value = 0)

for index in mean_stories.index:
    property_data.loc[(property_data["STORIES"]==0)&(property_data["TAXCLASS"] == index), "STORIES"] = mean_stories[index]

In [48]:
#filling in missing fullval, avland, avtot

property_data['FULLVAL'].replace("NaN",0)
temp = property_data[property_data["FULLVAL"]!= 0]
mean_fullval = temp.groupby("TAXCLASS") ["FULLVAL"].mean()
print(mean_fullval)

TAXCLASS
1     5.698435e+05
1A    3.352842e+05
1B    5.613639e+05
1C    7.615359e+05
1D    2.233614e+07
2     7.998018e+05
2A    8.640037e+05
2B    1.252989e+06
2C    7.728799e+05
3     1.112765e+05
4     3.211928e+06
Name: FULLVAL, dtype: float64


In [49]:
property_data['AVLAND'].replace('NaN', 0)
temp_avland = property_data[property_data['AVLAND']!=0]
mean_avland = temp_avland.groupby('TAXCLASS')['AVLAND'].mean()

In [50]:
property_data['AVTOT'].replace('NaN', 0)
temp_avtot = property_data[property_data['AVTOT']!=0]
mean_avtot = temp_avland.groupby('TAXCLASS')['AVTOT'].mean()

In [59]:
%%time
for index in mean_fullval.index:
    property_data.loc[(property["FULLVAL"]==0)&(property_data['TAXCLASS']==index), "FULLVAL"] = mean_fullval[index]
    property_data.loc[(property["AVLAND"]==0)&(property_data['TAXCLASS']==index), "AVLAND"] = mean_fullval[index]
    property_data.loc[(property["AVTOT"]==0)&(property_data['TAXCLASS']==index), "AVTOT"] = mean_fullval[index]

TypeError: 'type' object is not subscriptable

In [56]:
# Filling In Missing Lot and Building Sizes

# as these 4 values do not have NAs, we just need to replace 0s. 
# calculate groupwise average. 1st replace 0 by NAs so they are not counted in calculating mean.
property_data.loc[property_data['LTFRONT']==0,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==0,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==0,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan

# calculate mean now (mean function ignores NAs but not 0s hence we converted 0 to NA)
mean_LTFRONT=property_data.groupby(property_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH=property_data.groupby(property_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT=property_data.groupby(property_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH=property_data.groupby(property_data['TAXCLASS'])['BLDDEPTH'].mean()

# impute values
for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull())&(property_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]    
    property_data.loc[(property_data['LTDEPTH'].isnull())&(property_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull())&(property_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull())&(property_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]

In [68]:
#making ny varibales

data = property_data

data.head()

data["ZIP"] = data["ZIP"].astype(str)
data["zip3"] = data["ZIP"].str[:3]

data["ltsize"] = data['LTFRONT']*data['LTDEPTH']
data["bldsize"] = data['BLDFRONT']*data['BLDDEPTH']
data["bldvol"] = data['bldsize']*data['STORIES']

data["r1"] = data["FULLVAL"]/data['ltsize']
data["r2"] = data["FULLVAL"]/data['bldsize']
data["r3"] = data["FULLVAL"]/data['bldvol']



data["r4"] = data["AVLAND"]/data['ltsize']
data["r5"] = data["AVLAND"]/data['bldsize']
data["r6"] = data["AVLAND"]/data['bldvol']

data["r7"] = data["AVTOT"]/data['ltsize']
data["r8"] = data["AVTOT"]/data['bldsize']
data["r9"] = data["AVTOT"]/data['bldvol']

ninevars = ['r1','r2','r3','r4','r5','r6','r7','r8','r9',]


zip5_mean = data.groupby("ZIP")[ninevars].mean()
zip3_mean = data.groupby("zip3")[ninevars].mean()
taxclass_mean = data.groupby("TAXCLASS")[ninevars].mean()
borough_mean = data.groupby("B")[ninevars].mean()

data = data.join(zip5_mean, on = 'ZIP', rsuffix = '_zip5')
data = data.join(zip3_mean, on = 'zip3', rsuffix = '_zip3')
data = data.join(taxclass_mean, on = 'TAXCLASS', rsuffix = '_taxclass')
data = data.join(borough_mean, on = 'B', rsuffix = '_boro')
rsuffix = ['_zip5', '_zip3','_taxclass', '_boro' ]

for var in ninevars:
    for r in rsuffix:
        data[str(var)+r] = data[var] / data[str(var)+r]

In [70]:
list(data.columns) 

['RECORD',
 'BBLE',
 'B',
 'BLOCK',
 'LOT',
 'EASEMENT',
 'OWNER',
 'BLDGCL',
 'TAXCLASS',
 'LTFRONT',
 'LTDEPTH',
 'EXT',
 'STORIES',
 'FULLVAL',
 'AVLAND',
 'AVTOT',
 'EXLAND',
 'EXTOT',
 'EXCD1',
 'STADDR',
 'ZIP',
 'EXMPTCL',
 'BLDFRONT',
 'BLDDEPTH',
 'AVLAND2',
 'AVTOT2',
 'EXLAND2',
 'EXTOT2',
 'EXCD2',
 'PERIOD',
 'YEAR',
 'VALTYPE',
 'zip3',
 'ltsize',
 'bldsize',
 'bldvol',
 'r1',
 'r2',
 'r3',
 'r4',
 'r5',
 'r6',
 'r7',
 'r8',
 'r9',
 'r1_zip5',
 'r2_zip5',
 'r3_zip5',
 'r4_zip5',
 'r5_zip5',
 'r6_zip5',
 'r7_zip5',
 'r8_zip5',
 'r9_zip5',
 'r1_zip3',
 'r2_zip3',
 'r3_zip3',
 'r4_zip3',
 'r5_zip3',
 'r6_zip3',
 'r7_zip3',
 'r8_zip3',
 'r9_zip3',
 'r1_taxclass',
 'r2_taxclass',
 'r3_taxclass',
 'r4_taxclass',
 'r5_taxclass',
 'r6_taxclass',
 'r7_taxclass',
 'r8_taxclass',
 'r9_taxclass',
 'r1_boro',
 'r2_boro',
 'r3_boro',
 'r4_boro',
 'r5_boro',
 'r6_boro',
 'r7_boro',
 'r8_boro',
 'r9_boro']

In [83]:
newdata = data.drop(columns=['RECORD','BBLE','B','BLOCK',
 'LOT',
 'EASEMENT',
 'OWNER',
 'BLDGCL',
 'TAXCLASS',
 'LTFRONT',
 'LTDEPTH',
 'EXT',
 'STORIES',
 'FULLVAL',
 'AVLAND',
 'AVTOT',
 'EXLAND',
 'EXTOT',
 'EXCD1',
 'STADDR',
 'ZIP',
 'EXMPTCL',
 'BLDFRONT',
 'BLDDEPTH',
 'AVLAND2',
 'AVTOT2',
 'EXLAND2',
 'EXTOT2',
 'EXCD2',
 'PERIOD',
 'YEAR',
 'VALTYPE',
 'zip3',
 'ltsize',
 'bldsize',
 'bldvol'])


newdata.head()

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r1_zip5,...,r9_taxclass,r1_boro,r2_boro,r3_boro,r4_boro,r5_boro,r6_boro,r7_boro,r8_boro,r9_boro
0,40.917782,3784.190253,694.733351,8.079350,747.200744,137.177373,18.413002,1702.885614,312.630008,0.101284,...,2.645121,0.116695,6.927330,5.029897,0.243239,11.883148,6.138885,0.204666,11.960149,9.188294
1,53655.384245,34269.909866,6291.557172,3961.860416,2530.456193,464.562348,24144.922910,15421.459440,2831.200728,132.813795,...,23.954412,153.021428,62.734417,45.551124,119.276815,40.243248,20.789833,268.377879,108.312000,83.209878
2,261.796157,261.796157,87.265386,97.551991,97.551991,32.517330,117.808271,117.808271,39.269424,0.648027,...,0.332253,0.746624,0.479243,0.631805,2.936926,1.551423,1.455197,1.309473,0.827422,1.154141
3,89.714219,836.980890,418.490445,34.913021,325.717946,162.858973,40.371399,376.641401,188.320700,0.222071,...,1.593356,0.255859,1.532175,3.029887,1.051101,5.180073,7.288173,0.448740,2.645326,5.534804
4,669.074647,53676.325646,53676.325646,297.434763,23861.620343,23861.620343,301.083591,24154.346541,24154.346541,1.656168,...,204.366706,1.908154,98.259756,388.618733,8.954649,379.484583,1067.841799,3.346632,169.647081,709.903824


In [85]:
stats = newdata.describe().transpose()

stats

,count,mean,std,min,25%,50%,75%,max
r1,1070994.0,205.791067,1007.578972,0.0,68.250000,149.095634,239.473684,8.360000e+05
r2,1070994.0,572.751206,7756.901696,0.0,158.995202,491.612689,680.000000,3.712000e+06
r3,1070994.0,260.957063,3754.310459,0.0,58.914786,234.722222,333.007812,2.930000e+06
r4,1070994.0,9.854869,168.422266,0.0,2.111500,4.493176,7.135088,1.540706e+05
r5,1070994.0,33.342159,2424.086920,0.0,4.925867,14.704503,20.106761,1.549530e+06
r6,1070994.0,15.597354,1437.992795,0.0,1.426355,7.004545,9.937727,1.318500e+06
r7,1070994.0,24.249175,215.794104,0.0,5.011448,8.381053,13.270000,1.540706e+05
r8,1070994.0,62.828799,3066.275144,0.0,15.857143,26.430921,35.505314,1.670400e+06
r9,1070994.0,24.999117,1486.602940,0.0,6.046862,12.462273,17.371429,1.318500e+06
r1_zip5,1070994.0,1.000000,3.615980,0.0,0.435288,0.930221,1.252396,2.670100e+03


In [86]:
stats.to_csv("stats_on_vars.csv")